# Session and Study Similarity
* Session similarity for all five studies
* Study Similarity
    * Fall 2017 to Fall 2018 (Cross-Sample)
    * Spring 2018 to Spring 2019 to Fall 2019 (Cross-Sample)
    * Fall 2017 to Spring 2018 (Cross-Setting)
    * Fall 2018 to Spring 2019 (Cross-Setting)
* Each pre-processing technique receives its own df

In [1]:
import pandas as pd
from scipy import spatial
from openpyxl import load_workbook
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
clean_filepath = "/Users/kylieleblancKylie/domino/docsim/data/clean/"
table_filepath = "/Users/kylieleblancKylie/domino/docsim/results/"

clean_filepath = "/Users/kylie/docsim/data/clean/"
table_filepath = "/Users/kylie/docsim/results/"

In [3]:
docs = pd.read_csv(clean_filepath + 'text_transcripts.csv')
docs = docs.set_index('doc')
docs.sample(5)

,text,year,semester,scenario
doc,,,,
2018_44_3C_Transcript.docx,"So, I’m supposed to ask you how you feel but...",2018-19,fall,feedback
71_c_Transcript.docx,So what do you think? What are your though...,2017-18,fall,feedback
45_c_Transcript.docx,"How do you feel? Okay, all right. Let’s...",2017-18,fall,feedback
68_c_Transcript.docx,Nice. What did you think? Yeah? What ...,2017-18,fall,feedback
01_1920_05_097_22c_Transcript.docx,How do you feel? Do you know anybody who tea...,2019-20,fall,behavior


In [147]:
# file = table_filepath + 'Spring 2018 Results.xlsx'
# wb = load_workbook(file)
# ws = wb.active

In [148]:
def distance_to_mean(matrix_main, matrix_comparison):
    sims = []
    for maindoc in matrix_main.index:
        matrix_comp = matrix_comparison[~matrix_comparison.index.isin([maindoc])] # exclude self
        centroid = np.mean(matrix_comp)
        sim = 1 - spatial.distance.cosine(matrix_main.loc[maindoc], centroid)
        sims.append(sim)    
    return sims

# Session Similarity

In [149]:
matrix_transcripts = pd.read_csv(clean_filepath + 'matrix_transcripts.csv')
matrix_transcripts = matrix_transcripts.set_index('doc')
results = docs[(docs.year == '2017-18') & (docs.semester == 'fall')]
matrix = matrix_transcripts[matrix_transcripts.index.isin(results.index)]
sims = distance_to_mean(matrix, matrix)
results['session_sim'] = sims
semesters = ['spring', 'fall', 'spring', 'fall']
years = ['2017-18', '2018-19', '2018-19', '2019-20']
for semester, year in zip(semesters, years):
    df = docs[(docs.year == year) & (docs.semester == semester)]
    matrix = matrix_transcripts[matrix_transcripts.index.isin(df.index)]
    sims = distance_to_mean(matrix, matrix)
    df['session_sim'] = sims
    results = results.append(df)
results

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,text,year,semester,scenario,session_sim
doc,,,,,
68_c_Transcript.docx,Nice. What did you think? Yeah? What ...,2017-18,fall,feedback,0.869201
90_c_Transcript.docx,Maybe I didn’t turn the volume up enough. I...,2017-18,fall,feedback,0.899263
122_c_Transcript.docx,"Yeah, and don’t be afraid, seriously, if lik...",2017-18,fall,feedback,0.864731
12_c_Transcript.docx,"So, what are some things that you feel like...",2017-18,fall,feedback,0.905865
113_c_Transcript.docx,"Like what do you think went well, in terms ...",2017-18,fall,feedback,0.863749
...,...,...,...,...,...
01_1920_05_005_22c_Transcript.docx,"This is Arielle Bogaslav, Bogaslav’s coachi...",2019-20,fall,behavior,0.909029
01_1920_05_052_22c_Transcript.docx,"So first, how'd you feel? Which is tota...",2019-20,fall,behavior,0.925759
01_1920_05_051_22c_Transcript.docx,"This is hard. Alright, well why don’t you...",2019-20,fall,behavior,0.944563


# Study Similarity
    * Fall 2017 to Fall 2018 (Cross-Sample)
    * Spring 2018 to Spring 2019 to Fall 2019 (Cross-Sample)
    * Fall 2017 to Spring 2018 (Cross-Setting)
    * Fall 2018 to Spring 2019 (Cross-Setting)

In [133]:
semesters = ['fall', 'spring', 'fall', 'spring', 'fall']
years = ['2017-18', '2017-18', '2018-19', '2018-19', '2019-20']

matrix_transcripts = pd.read_csv(clean_filepath + 'matrix_transcripts.csv')
matrix_transcripts = matrix_transcripts.set_index('doc')

for semester, year in zip(semesters, years):
    results_study = docs
    matrix = matrix_transcripts

    comp = docs[(docs.year == year) & (docs.semester == semester)]
    matrix_comp = matrix_transcripts[matrix_transcripts.index.isin(comp.index)]

    sims = distance_to_mean(matrix, matrix_comp)
    results['study_sim_' + semester + year] = sims
results.sample(5)

,text,year,semester,scenario,session_sim,study_sim_fall2017-18,study_sim_spring2017-18,study_sim_fall2018-19,study_sim_spring2018-19,study_sim_fall2019-20
doc,,,,,,,,,,
27_c_Transcript.docx,So how do you think it went? What went wel...,2017-18,fall,feedback,0.905977,0.905977,0.861025,0.842855,0.864747,0.842803
89_c_Transcript.docx,How do you think that went? Oh! what do...,2017-18,fall,feedback,0.898512,0.898512,0.828248,0.854511,0.851975,0.821918
68_c_Transcript.docx,Nice. What did you think? Yeah? What ...,2017-18,fall,feedback,0.869201,0.869201,0.797505,0.797829,0.825035,0.824781
2019_71_5C_Transcript.docx,Okay so how are you feeling about the first...,2018-19,spring,behavior,0.911177,0.871307,0.860377,0.890953,0.911177,0.884880
32_c_Transcript.docx,How do you feel like – what do you think we...,2017-18,fall,feedback,0.904372,0.904372,0.862142,0.845583,0.886772,0.879849


# Script Similarity

In [134]:
def distance_to_doc(matrix_main, comp_doc):
    """
    Returns list of similarity of every doc in matrix_main to comp_doc
    """
    sims = []
    for maindoc in matrix_main.index:
        sim = 1 - spatial.distance.cosine(matrix_main.loc[maindoc], comp_doc)
        sims.append(sim)    
    return sims

In [135]:
matrix_transcripts = pd.read_csv(clean_filepath + 'matrix_transcripts.csv')
matrix_transcripts = matrix_transcripts.set_index('doc')


matrix_scripts = pd.read_csv(clean_filepath + 'matrix_scripts.csv')
matrix_scripts = matrix_scripts.set_index('doc')
matrix_scripts

,00,04,05,10,100,12,13,15,16,18,...,young,younger,your,yours,yourself,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
Feedback Model C22.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Feedback Model D21.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0
Feedback Model A22.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Feedback Model B11.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
Feedback Model C12.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Feedback Model D11.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0
Feedback Model A12.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Feedback Model B21.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
Feedback Model B22.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


In [136]:
feedback_results = docs[(docs.scenario == 'feedback')]
feedback_matrix = matrix_transcripts[matrix_transcripts.index.isin(feedback_results.index)]
feedback_scripts = matrix_scripts[matrix_scripts.index.str.contains('Feedback')]

script_indices = []
script_names = []
for doc in feedback_scripts.index:
    script_indices.append(str(doc))
    script_names.append(str(doc[15:18]))

for script, col in zip(script_indices, script_names):
    sims = distance_to_doc(feedback_matrix, feedback_scripts.loc[script])
    feedback_results[col] = sims
    
feedback_results['script_sim'] = feedback_results[script_names].max(axis=1)
feedback_results.sample(5)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,text,year,semester,scenario,C22,D21,A22,B11,C12,D11,...,B21,B22,D12,A11,C11,B12,D22,A21,C21,script_sim
doc,,,,,,,,,,,,,,,,,,,,,
104_c_Transcript.docx,So how do you think that went? Yeah. ...,2017-18,fall,feedback,0.736012,0.675909,0.625883,0.665920,0.737078,0.678291,...,0.663890,0.667027,0.682328,0.621925,0.730528,0.665920,0.679908,0.619958,0.729495,0.737078
57_c_Transcript.docx,All right. Nice job Taylor. How do you ...,2017-18,fall,feedback,0.643027,0.595425,0.575627,0.638109,0.643959,0.601527,...,0.628648,0.628243,0.600864,0.582592,0.642570,0.638109,0.594700,0.575601,0.641662,0.643959
2018_80_3C_Transcript.docx,"So, I would love to hear what – how did you...",2018-19,fall,feedback,0.723425,0.729251,0.716257,0.770036,0.723467,0.727244,...,0.773722,0.771464,0.724160,0.715055,0.724746,0.770036,0.726180,0.718912,0.724715,0.773722
75_c_Transcript.docx,"How do you feel? Good, okay. So what a...",2017-18,fall,feedback,0.622634,0.610868,0.584043,0.619949,0.623536,0.612640,...,0.615348,0.615079,0.612358,0.584825,0.622099,0.619949,0.610561,0.583537,0.621220,0.623536
33_c_Transcript.docx,Yeah. So how do you think that went? D...,2017-18,fall,feedback,0.678105,0.668228,0.642639,0.651638,0.679088,0.667968,...,0.651907,0.653704,0.670380,0.637004,0.674769,0.651638,0.670629,0.638476,0.673815,0.679088


In [137]:
behavior_results = docs[(docs.scenario == 'behavior')]
behavior_matrix = matrix_transcripts[matrix_transcripts.index.isin(behavior_results.index)]
behavior_scripts = matrix_scripts[matrix_scripts.index.str.contains('Classroom Management')]

script_indices = []
script_names = []
for doc in behavior_scripts.index:
    script_indices.append(str(doc))
    script_names.append(str(doc[27:29]))

for script, col in zip(script_indices, script_names):
    sims = distance_to_doc(behavior_matrix, behavior_scripts.loc[script])
    behavior_results[col] = sims
behavior_results['script_sim'] = behavior_results[script_names].max(axis=1)
behavior_results

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,text,year,semester,scenario,D1,A6,C6,B1,B6,A1,...,C3,A4,D3,C4,B2,B3,C5,D2,A5,script_sim
doc,,,,,,,,,,,,,,,,,,,,,
113-2C.docx,"Yeah, how are you feeling about that? Well...",2017-18,spring,behavior,0.666223,0.608953,0.668305,0.672449,0.668933,0.613354,...,0.672374,0.612645,0.669314,0.672731,0.671454,0.671454,0.672863,0.668564,0.613354,0.672863
103-2C.docx,All right. Why don't you have a sit and we'l...,2017-18,spring,behavior,0.603628,0.533082,0.577155,0.608986,0.605075,0.537359,...,0.581672,0.537359,0.604576,0.581981,0.608497,0.608497,0.583592,0.605921,0.537359,0.610516
93-2C.docx,"Come on I said Um, so how are you feeling? ...",2017-18,spring,behavior,0.644356,0.597129,0.656855,0.649930,0.650057,0.596012,...,0.654770,0.595683,0.648512,0.655117,0.650515,0.650515,0.655972,0.648305,0.596012,0.656855
83-2C.docx,So how do you feel about that one? Way har...,2017-18,spring,behavior,0.677166,0.613284,0.658505,0.672945,0.668083,0.618778,...,0.664455,0.618778,0.681503,0.664807,0.672089,0.672089,0.664809,0.681318,0.618778,0.681503
6-2C.docx,So how do you feel about that? Really hard...,2017-18,spring,behavior,0.679941,0.664689,0.692821,0.694641,0.692510,0.666571,...,0.694197,0.664887,0.682241,0.694565,0.694211,0.694211,0.694512,0.681756,0.666571,0.694713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01_1920_05_005_22c_Transcript.docx,"This is Arielle Bogaslav, Bogaslav’s coachi...",2019-20,fall,behavior,0.798251,0.742132,0.795068,0.809903,0.808987,0.742169,...,0.794947,0.742169,0.795436,0.795369,0.810632,0.810632,0.797774,0.794870,0.742169,0.812589
01_1920_05_052_22c_Transcript.docx,"So first, how'd you feel? Which is tota...",2019-20,fall,behavior,0.730704,0.685303,0.731256,0.757573,0.756431,0.686986,...,0.732225,0.686665,0.731346,0.732613,0.756562,0.756562,0.735323,0.732315,0.686986,0.760145
01_1920_05_051_22c_Transcript.docx,"This is hard. Alright, well why don’t you...",2019-20,fall,behavior,0.756435,0.685824,0.752111,0.778047,0.773602,0.690868,...,0.757365,0.690868,0.757425,0.757767,0.777132,0.777132,0.759714,0.759491,0.690868,0.779930


In [138]:
script_results = feedback_results[['script_sim']]
script_results = script_results.append(behavior_results[['script_sim']])
results = results.merge(script_results, left_index = True, right_index = True)
results.to_csv((clean_filepath + 'results.csv'))